# GMM

## 0. Imports

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.optimize as opt
from utils import noise_generators
from utils import simulate as sim
import matplotlib.pyplot as plt
from scipy.stats import logistic
import functools

In [ ]:

functions = []
def f(i, x):
    return i * x

for i in range(3):    
    f_with_i = functools.partial(f, i)  # important: use a different variable than "f"
    functions.append(f_with_i)

In [ ]:
functions[1](2)

In [2]:
data_df = data_df[["Y", "intercept", "X"]]


NameError: name 'data_df' is not defined

In [ ]:
data = data_df.values

In [ ]:
data.shape[1]

In [3]:
def linear_regression_moments(data, theta, noise=False, DP_epsilon=None):
    # 1st column = Y
    output = np.zeros(data.shape[1] - 1) # initialization
    Y = data[:, 0]
    X = data[:, 1:]
    basis = Y - X.dot(theta)
    for k in range(X.shape[1]):
        output[k] = np.mean(np.multiply(basis, X[:, k]))
        if noise:
            sensitivity = 0.1
            scale = sensitivity / DP_epsilon
            output[k] = output[k] + np.mean(np.random.default_rng().laplace(0, scale, data.shape[0])) 
    return output

In [4]:
np.mean(np.random.default_rng().laplace(0, 10, 1000))

0.8613369430766279

In [5]:
def GMM_objective(theta, *args):
    data_df, f = args
    moments = f(data_df.values, theta, True, 10)
    return (moments ** 2).sum()

In [6]:
params_init = np.array([0.0, 0.0])
results = opt.minimize(GMM_objective, params_init, args=(data_df, linear_regression_moments), method='L-BFGS-B', bounds=((1e-10, None), (1e-10, None)))
results.x

NameError: name 'data_df' is not defined

In [ ]:
results.nit

In [ ]:
results.success

In [ ]:
results.message

In [ ]:
results.hess_inv()

In [ ]:
(np.array([1,1])))

In [ ]:
data_df.apply(lambda row: linear_regression_moments(data_df.loc[0,:], "Y", "X")(np.array([1,1])))

In [ ]:
range()

In [ ]:
for f in linear_regression_moments(data_df.loc[0,:], "Y", "X"):
    print(f(np.array([0,0])))
data_df.mean()

In [ ]:
data_df.head(3)

In [ ]:
n = 1000
variance_X = 3
mu_X = 4
alpha = 2
beta = .5
desired_R_squared = .7
data_df = sim.univariate_linear_regression(mu_X, variance_X, alpha, beta, desired_R_squared, n)
data_df.head(3)

In [ ]:
np.linspace(0, 4, 1)

In [ ]:
ols_results = smf.ols("Y ~ 1 + X", data=data_df).fit()
print(ols_results.summary())

In [ ]:
linear_regression_moments(params_init, data_df.loc[0,:], "Y", "X")

In [ ]:
np.array(params_init[1]).dot(data_df.loc[0, "X"])

In [ ]:
(data_df.apply(lambda row: function_moment_condition(theta, row, "Y", "X"), axis=1).mean()) ** 2

In [ ]:
np.random.default_rng(123).laplace(0, 1)

In [ ]:
def LDP_moment_condition(list_functions_moment_conditions, theta, data_df, epsilon):
    sensitivity = 5
    seed = 123
    list_moments = []
    n = len(data_df)
    scale = sensitivity / epsilon
    for function_moment_condition in list_functions_moment_conditions:
        output = 0
        contrib_moment_true = data_df.apply(lambda row: function_moment_condition(theta, row), axis=1)
        contrib_moment_LDP = contrib_moment_true #+ np.random.default_rng(seed).laplace(0, scale, contrib_moment_true.shape)
        output = contrib_moment_LDP.mean()
        list_moments.append(output)
    return list_moments

In [ ]:
def moment1(theta, row):
    return row[0] - theta[0] - theta[1] * row[1]
def moment2(theta, row):
    return (row[0] - theta[0] - theta[1] * row[1]) * row[1]

In [ ]:
ols_results = smf.ols("Y ~ 1 + X", data=data_df).fit()
print(ols_results.summary())

In [ ]:
a =  LDP_moment_condition([moment1, moment2], np.array([0,0]), data_df, 1)

In [ ]:
data_df["pert"] = a[0]
data_df["pert2"] = a[1]
data_df
4.33 * 4.77

## Logistic regression

In [ ]:
n = 1000
variance_X = 1
mu_X = 1
X = np.random.normal(mu_X, np.sqrt(variance_X), n)

In [ ]:
Xext = np.vstack([np.ones(len(X)), X]).T

In [ ]:
a = pd.DataFrame(Xext)

In [ ]:
beta = np.array([-1, 1.2])

In [ ]:
beta

In [ ]:
Y.shape

In [ ]:
p_X = logistic.cdf(a.apply(lambda row: row.dot(beta), axis=1))
Y = np.random.binomial(1., p_X)

pd.Series(p_X, index=X).plot(style='*')
#pd.Series(p_X, index=X).plot(style='.')
pd.Series(Y, index=X).plot(style='.')


In [ ]:


logistic()

In [ ]:
def logistic(x, b, noise=None):
    L = x.T.dot(b)
    if noise is not None:
        L = L+noise
    return 1/(1+np.exp(-L))

x = np.arange(-10., 10, 0.05)
bias = np.ones(len(x))
X = np.vstack([x,bias]) # Add intercept
B =  [1., 1.] # Sigmoid params for X

# True mean
p = logistic(X, B)
# Noisy mean
pnoisy = logistic(X, B, noise=np.random.normal(loc=0., scale=1., size=len(x)))
# dichotomize pnoisy -- sample 0/1 with probability pnoisy
dichot = np.random.binomial(1., pnoisy)

pd.Series(p, index=x).plot(style='-')
pd.Series(pnoisy, index=x).plot(style='.')
pd.Series(dichot, index=x).plot(style='.')

In [ ]:
dichot.shape

In [ ]:
data_df = pd.DataFrame(Y, columns=["Y"])

In [ ]:
data_df["X"] = X

In [ ]:
data_df

In [ ]:
data_df

In [ ]:
LDP_moment_condition([moment1, moment2], np.array([0,0]), data_df, 1)

In [ ]:
data_df = pd.DataFrame(columns=["Y", "X"])

In [ ]:
data_df["Y"] = dichot

In [ ]:
data_df

In [ ]:
def moment_logistic_regression_basis(theta, row):
    return row[0] - logistic.cdf(row.dot(theta))
def moment_logistic_regression_cov(theta, row):
    return moment_logistic_regression_basis(theta, row) * row[1]


In [ ]:
logistic(data_df.loc[0,:], params_init)

In [ ]:
moment_logistic_regression_basis(params_init, data_df.loc[0,:])

In [ ]:
params_init = np.array([1, 1])
results = opt.minimize(GMM_objective, params_init, args=(data_df, [moment_logistic_regression_basis, moment_logistic_regression_cov], 1), method='L-BFGS-B', bounds=((1e-10, None), (1e-10, None)))
results.x

In [ ]:
data_df["ones"] = 1

In [ ]:
logit = statsmodels.discrete.discrete_model.Logit(data_df["Y"], data_df[["ones", "X"]], data=data_df)

In [ ]:
import statsmodels

In [ ]:
logit.fit().summary()

# 1. Simulate dataset

We simulate a dataset according to a univariate linear regression model $Y=\alpha + \beta X + \varepsilon$. The regressor $X\sim N(\mu_X, \sigma^2)$ and the innovation $\epsilon$ is drawn (independently from $X$) from a Logistic distribution with mean zero. The variance is set in such a manner that the $R^2$ in the population, $R^2= \beta^2 \operatorname{var}(X)/(\beta^2 \operatorname{var}(X) + \operatorname{var}(\varepsilon))$, is equal to the specified value.

In [ ]:
n = 1000
variance_X = 3
mu_X = 4
alpha = 2
beta = .5
desired_R_squared = .7
data_df = sim.univariate_linear_regression(mu_X, variance_X, alpha, beta, desired_R_squared, n)

In [ ]:
data_df.head(5)

In [ ]:
data_df.plot.scatter(x="X", y="Y")

## 2. Fit OLS on 'true' dataset

Using statsmodels we estimate the parameters using OLS. Please compare the parameter estimates and the R-squares to the specification above.

In [ ]:
ols_results = smf.ols("Y ~ 1 + X", data=data_df).fit()
print(ols_results.summary())

## 3. Apply standard Local Differential Privacy using Laplace mechanism

In [ ]:
sensitivity = 15
epsilon = 10

In [ ]:
private_data_df = noise_generators.add_noise_laplace_mechanism(data_df, epsilon, 
                                sensitivity, seed=123)

## 4. Using OLS on the locally private data does not work!

In [ ]:
# Fit regression model 
ols_results = smf.ols("Y ~ 1 + X", data=private_data_df).fit()
print(ols_results.summary())

# 5. Using the proposed Local Differentially Private Framework I for GMM

For linear regression we can use two moment conditions:  $0=\mathbb{E}[ Y -\alpha -\beta X]$ and $0=\mathbb{E}[ XY -\alpha X -\beta X^2]$. These moment conditions can be solved analytically. We need to receive data on $Y$, $X$, $XY$, and $X^2$.

We first add the additional columns $XY$ and $X^2$ to the true dataset:

In [ ]:
data_df["X * Y"] = data_df["X"] * data_df["Y"]
data_df["X^2"] = data_df["X"] * data_df["X"]
data_df.head(5)

Now we apply the proposed framework for Local Differentially Privacy:

In [ ]:
private_data_df = noise_generators.add_noise_laplace_mechanism(data_df, epsilon, sensitivity, seed=123)
private_data_df.head(5)

Next we apply our moment estimator. First on the true data (which yields the same outputs as statsmodels) and after that on the locally differentially private dataset.

In [ ]:
def linear_regression_MM(Y, X, X_times_Y, X_squared):
    hat_beta = (X_times_Y.mean() - X.mean() * Y.mean()) / (X_squared.mean() - X.mean() ** 2)
    hat_alpha = Y.mean() - hat_beta * X.mean()
    return hat_alpha, hat_beta

In [ ]:
 linear_regression_MM(data_df["Y"], data_df["X"], data_df["X * Y"], data_df["X^2"])

In [ ]:
 linear_regression_MM(private_data_df["Y"], private_data_df["X"], private_data_df["X * Y"], private_data_df["X^2"])

We see that the results are quite close. This in sharp contrast to Section 4!